In [1]:
import cv2
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical

# Caminhos para os dados de treino
#path_normal_train = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/train/NORMAL/*.jpeg"
#path_pneumonia_train = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/train/PNEUMONIA/*.jpeg"
path_total = "C:/Users/Artur Vinicius/Documents/Python/chest_xray_v4/total_data/*.jpeg"
# Caminhos para os dados de teste
#path_normal_test = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/test/NORMAL/*.jpeg"
#path_pneumonia_test = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/test/PNEUMONIA/*.jpeg"

# Dimensões desejadas para as imagens
width = 128
height = 128
dim = (width, height)

def load_image(path):
    images = []
    for file in glob.glob(path):
        image = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image, dim)
        images.append(resized_image/255)
    return images


In [2]:
x_data = load_image(path_total)

In [10]:
print(x_data[1])

[[0.14901961 0.12941177 0.10588235 ... 0.48235294 0.45490196 0.42745098]
 [0.14901961 0.13333334 0.10588235 ... 0.4862745  0.45882353 0.4117647 ]
 [0.16470589 0.12156863 0.10196079 ... 0.47843137 0.44313726 0.4117647 ]
 ...
 [0.16862746 0.14117648 0.14901961 ... 0.1254902  0.13725491 0.12156863]
 [0.16470589 0.16862746 0.16078432 ... 0.11372549 0.13333334 0.13725491]
 [0.15294118 0.12941177 0.16862746 ... 0.1254902  0.12941177 0.11372549]]


In [3]:
y_data = [] # Target
string_bacteria = "bacteria"
string_virus = "virus"

for file_path in glob.glob(path_total):
    file_name = file_path.split("/")[-1]  # Extrair apenas o nome do arquivo
    if (string_bacteria in file_name) or (string_virus in file_name):
        y_data.append(1) # Rotular os RX com Pneumonia Bacteriana com 1
    #lif string_virus in file_name:
       # y_data.append(2) # Rotular os RX com Pneumonia Viral com 2
    elif (string_bacteria not in file_name) and (string_virus not in file_name):
        y_data.append(0) # Rotular os RX sem Pneumonia com o valor 0

In [4]:
x_data = np.asarray(x_data).astype(np.float32)
y_data = np.asarray(y_data).astype(np.float32)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.4, random_state=42)

In [19]:
# Definir o modelo
from keras.layers import MaxPooling2D
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(128,128,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Treinar o modelo
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
55/55 [==============================] - 14s 255ms/step - loss: 0.3847 - accuracy: 0.8301
Epoch 2/10
55/55 [==============================] - 14s 255ms/step - loss: 0.1695 - accuracy: 0.9362
Epoch 3/10
55/55 [==============================] - 14s 251ms/step - loss: 0.1597 - accuracy: 0.9365
Epoch 4/10
55/55 [==============================] - 14s 245ms/step - loss: 0.1290 - accuracy: 0.9510
Epoch 5/10
55/55 [==============================] - 14s 249ms/step - loss: 0.1313 - accuracy: 0.9522
Epoch 6/10
55/55 [==============================] - 14s 249ms/step - loss: 0.1187 - accuracy: 0.9582
Epoch 7/10
55/55 [==============================] - 14s 252ms/step - loss: 0.1096 - accuracy: 0.9576
Epoch 8/10
55/55 [==============================] - 14s 250ms/step - loss: 0.0928 - accuracy: 0.9670
Epoch 9/10
55/55 [==============================] - 14s 249ms/step - loss: 0.0819 - accuracy: 0.9695
Epoch 10/10
55/55 [==============================] - 14s 250ms/step - loss: 0.0983 - accura

In [21]:
#y_test_categorical = to_categorical(y_test)

y_pred = model.predict(X_test)

74/74 [==============================] - 4s 54ms/step


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.9590268886043534
Precision: 0.9458323289657271
Recall: 0.954568572084646
F1-score: 0.9500559552358114
